In [0]:
#Cargar archivo
ruta = "/FileStore/tables/access_log_Jul95"

base_df = spark.read.text(ruta)

base_df.show(10, truncate= False)

+-----------------------------------------------------------------------------------------------------------------------+
value |
+-----------------------------------------------------------------------------------------------------------------------+
199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085 |
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0 |
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0 |
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0 |
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985 |
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074 |
+-----------------------------------------------------------------------------------------------------------------------+
only showing top 10 rows

In [0]:
#hostnames 199.72.81.55 / unicomp6.unicomp.net
host=  '^(\S+)\s'

#timestamps 01/Jul/1995:00:00:01 -0400
ts_pattern =  '(\d{2}\/\w{3}\/\d{4}\:\d{2}:\d{2}:\d{2})'

#HTTP ‘GET’, ‘/history/apollo/’, ‘HTTP/1.0’
method_uri_protocol_pattern =  '\"(\S+)\s(\S+)\s*(\S*)\s*(\S*)\"'


#HTTP status codes 200
status_pattern =  '\s(\d{3})\s'


#HTTP response content size 6245
content_size_pattern =  '\s(\d+)$'


In [0]:
from pyspark.sql.functions import regexp_extract

#Putting it all together
logs_df=base_df.select(regexp_extract('value', host, 1).alias('host'),
                         regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                         regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                         regexp_extract('value', method_uri_protocol_pattern, 2).alias('resource'),
                         regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                         regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                         regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))


logs_df.show()

+--------------------+--------------------+------+--------------------+--------+------+------------+
 host| timestamp|method| resource|protocol|status|content_size|
+--------------------+--------------------+------+--------------------+--------+------+------------+
 199.72.81.55|01/Jul/1995:00:00:01| GET| /history/apollo/|HTTP/1.0| 200| 6245|
unicomp6.unicomp.net|01/Jul/1995:00:00:06| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 199.120.110.21|01/Jul/1995:00:00:09| GET|/shuttle/missions...|HTTP/1.0| 200| 4085|
 burger.letters.com|01/Jul/1995:00:00:11| GET|/shuttle/countdow...|HTTP/1.0| 304| 0|
 199.120.110.21|01/Jul/1995:00:00:11| GET|/shuttle/missions...|HTTP/1.0| 200| 4179|
 burger.letters.com|01/Jul/1995:00:00:12| GET|/images/NASA-logo...|HTTP/1.0| 304| 0|
 burger.letters.com|01/Jul/1995:00:00:12| GET|/shuttle/countdow...|HTTP/1.0| 200| 0|
 205.212.115.106|01/Jul/1995:00:00:12| GET|/shuttle/countdow...|HTTP/1.0| 200| 3985|
 d104.aa.net|01/Jul/1995:00:00:13| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 129.94.144.152|01/Jul/1995:00:00:13| GET| /|HTTP/1.0| 200| 7074|
unicomp6.unicomp.net|01/Jul/1995:00:00:14| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
unicomp6.unicomp.net|01/Jul/1995:00:00:14| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
unicomp6.unicomp.net|01/Jul/1995:00:00:14| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 d104.aa.net|01/Jul/1995:00:00:15| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
 d104.aa.net|01/Jul/1995:00:00:15| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
 d104.aa.net|01/Jul/1995:00:00:15| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 129.94.144.152|01/Jul/1995:00:00:17| GET|/images/ksclogo-m...|HTTP/1.0| 304| 0|
 199.120.110.21|01/Jul/1995:00:00:17| GET|/images/launch-lo...|HTTP/1.0| 200| 1713|
ppptky391.asahi-n...|01/Jul/1995:00:00:18| GET|/facts/about_ksc....|HTTP/1.0| 200| 3977|
 net-1-141.eden.com|01/Jul/1995:00:00:19| GET|/shuttle/missions...|HTTP/1.0| 200| 34029|
+--------------------+--------------------+------+--------------------+--------+------+------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import *


#Convertir el mes de la fecha, en número
logs_df1 = logs_df.withColumn("timestamp", to_timestamp(logs_df.timestamp, "dd/MMM/yyyy:HH:mm:ss"))

logs_df1.show()

+--------------------+-------------------+------+--------------------+--------+------+------------+
 host| timestamp|method| resource|protocol|status|content_size|
+--------------------+-------------------+------+--------------------+--------+------+------------+
 199.72.81.55|1995-07-01 00:00:01| GET| /history/apollo/|HTTP/1.0| 200| 6245|
unicomp6.unicomp.net|1995-07-01 00:00:06| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 199.120.110.21|1995-07-01 00:00:09| GET|/shuttle/missions...|HTTP/1.0| 200| 4085|
 burger.letters.com|1995-07-01 00:00:11| GET|/shuttle/countdow...|HTTP/1.0| 304| 0|
 199.120.110.21|1995-07-01 00:00:11| GET|/shuttle/missions...|HTTP/1.0| 200| 4179|
 burger.letters.com|1995-07-01 00:00:12| GET|/images/NASA-logo...|HTTP/1.0| 304| 0|
 burger.letters.com|1995-07-01 00:00:12| GET|/shuttle/countdow...|HTTP/1.0| 200| 0|
 205.212.115.106|1995-07-01 00:00:12| GET|/shuttle/countdow...|HTTP/1.0| 200| 3985|
 d104.aa.net|1995-07-01 00:00:13| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 129.94.144.152|1995-07-01 00:00:13| GET| /|HTTP/1.0| 200| 7074|
unicomp6.unicomp.net|1995-07-01 00:00:14| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
unicomp6.unicomp.net|1995-07-01 00:00:14| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
unicomp6.unicomp.net|1995-07-01 00:00:14| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 d104.aa.net|1995-07-01 00:00:15| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
 d104.aa.net|1995-07-01 00:00:15| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
 d104.aa.net|1995-07-01 00:00:15| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 129.94.144.152|1995-07-01 00:00:17| GET|/images/ksclogo-m...|HTTP/1.0| 304| 0|
 199.120.110.21|1995-07-01 00:00:17| GET|/images/launch-lo...|HTTP/1.0| 200| 1713|
ppptky391.asahi-n...|1995-07-01 00:00:18| GET|/facts/about_ksc....|HTTP/1.0| 200| 3977|
 net-1-141.eden.com|1995-07-01 00:00:19| GET|/shuttle/missions...|HTTP/1.0| 200| 34029|
+--------------------+-------------------+------+--------------------+--------+------+------------+
only showing top 20 rows

In [0]:
# ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.

logs_df1.select("protocol").distinct().show()

+--------------------+
 protocol|
+--------------------+
 align=left|
�?shuttle%20tracking|
 Shield|
hqpao/hqpao_home....|
 /|
 |
�?tracking%20station|
 @net.com|
 HTTP/V1.0|
 HTTP/1.0|
 /facts/facts.html|
 sts71|
 html|
 (31K)|
 13|
apollo/apollo-13/...|
apollo/apollo-13/...|
 13.html|
apollo/apollo-13/...|
apollo/apollo-13/...|
+--------------------+
only showing top 20 rows

In [0]:
#¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos  para ver cuál es el más común.

logs_df1.groupBy("status").count().show()

+------+-------+
status| count|
+------+-------+
 500| 62|
 403| 54|
 404| 10843|
 200|1701536|
 304| 132627|
 302| 46573|
 501| 14|
 400| 5|
 null| 1|
+------+-------+

In [0]:
#¿Y los métodos de petición (verbos) más utilizados?

logs_df1.groupBy("method").count().show()


+------+-------+
method| count|
+------+-------+
 POST| 111|
 HEAD| 3950|
 GET|1887605|
 | 49|
+------+-------+

In [0]:
#¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

logs_df1.groupBy("resource","content_size").sum("content_size").show()

+--------------------+------------+-----------------+
 resource|content_size|sum(content_size)|
+--------------------+------------+-----------------+
/history/apollo/a...| 193700| 15496000|
/history/apollo/a...| 3842| 7868416|
/history/mercury/...| 421| 197028|
/cgi-bin/imagemap...| 110| 440|
/cgi-bin/imagemap...| 110| 550|
/shuttle/countdow...| 65734| 65734|
/shuttle/countdow...| 7878| 811434|
/cgi-bin/imagemap...| 96| 2880|
/history/mercury/...| 20641| 9639347|
/shuttle/countdow...| 52569| 210276|
/cgi-bin/imagemap...| 114| 114|
/shuttle/countdow...| 45514| 45514|
/cgi-bin/imagemap...| 110| 4180|
/shuttle/missions...| 0| 0|
/shuttle/countdow...| 47900| 239500|
/history/apollo/a...| 113886| 5124870|
/cgi-bin/imagemap...| 110| 660|
/shuttle/missions...| 1430| 37180|
/cgi-bin/imagemap...| 110| 2090|
/shuttle/countdow...| 46170| 46170|
+--------------------+------------+-----------------+
only showing top 20 rows

In [0]:
#Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es  decir, el recurso con más registros en nuestro log

logs_df1.groupBy("resource").count().orderBy(col("count").desc()).show()

+--------------------+------+
 resource| count|
+--------------------+------+
/images/NASA-logo...|111388|
/images/KSC-logos...| 89639|
/images/MOSAIC-lo...| 60468|
/images/USA-logos...| 60014|
/images/WORLD-log...| 59489|
/images/ksclogo-m...| 58802|
/images/launch-lo...| 40871|
 /shuttle/countdown/| 40279|
 /ksc.html| 40229|
/images/ksclogosm...| 33585|
 /| 32910|
/history/apollo/i...| 31072|
/shuttle/missions...| 24864|
 /htbin/cdt_main.pl| 22626|
/shuttle/countdow...| 22216|
/shuttle/countdow...| 22000|
/shuttle/countdow...| 20957|
/images/launchmed...| 20812|
/shuttle/missions...| 19853|
/shuttle/missions...| 18159|
+--------------------+------+
only showing top 20 rows

In [0]:
#¿Qué días la web recibió más tráfico?

logs_df1.groupBy(dayofmonth("timestamp")).count().orderBy(col("count").desc()).show()

+---------------------+------+
dayofmonth(timestamp)| count|
+---------------------+------+
 13|134203|
 6|100960|
 5| 94575|
 12| 92536|
 3| 89584|
 7| 87233|
 14| 84103|
 11| 80407|
 17| 74981|
 10| 72860|
 19| 72738|
 4| 70452|
 20| 66593|
 1| 64714|
 21| 64629|
 18| 64282|
 24| 64259|
 25| 62699|
 27| 61680|
 2| 60265|
+---------------------+------+
only showing top 20 rows

In [0]:
# ¿Cuáles son los hosts los más frecuentes?

logs_df1.groupBy("host").count().orderBy(col("count").desc()).show()

+--------------------+-----+
 host|count|
+--------------------+-----+
piweba3y.prodigy.com|17572|
piweba4y.prodigy.com|11591|
piweba1y.prodigy.com| 9868|
 alyssa.prodigy.com| 7852|
 siltb10.orl.mmc.com| 7573|
piweba2y.prodigy.com| 5922|
 edams.ksc.nasa.gov| 5434|
 163.206.89.4| 4906|
 news.ti.com| 4863|
disarray.demon.co.uk| 4353|
www-a2.proxy.aol.com| 4179|
www-d1.proxy.aol.com| 4158|
 vagrant.vf.mmc.com| 4146|
 198.133.29.18| 4136|
 poppy.hensa.ac.uk| 4117|
www-d4.proxy.aol.com| 3977|
 webgate1.mot.com| 3843|
www-b3.proxy.aol.com| 3755|
 e659229.boeing.com| 3737|
www-b2.proxy.aol.com| 3732|
+--------------------+-----+
only showing top 20 rows

In [0]:
# ¿A qué horas se produce el mayor número de tráfico en la web?


logs_df1.groupBy(hour("timestamp")).count().orderBy(col("count").desc()).show()

+---------------+------+
hour(timestamp)| count|
+---------------+------+
 14|122479|
 12|122085|
 15|121200|
 13|120814|
 16|118037|
 11|115720|
 10|105507|
 9| 99969|
 17| 97609|
 8| 83750|
 18| 79282|
 21| 71922|
 19| 71776|
 22| 70759|
 20| 69809|
 23| 69362|
 0| 62450|
 7| 54017|
 1| 53066|
 2| 45297|
+---------------+------+
only showing top 20 rows

In [0]:
# ¿Cuál es el número de errores 404 que ha habido cada día?

logs_df1.groupBy(dayofmonth(col("timestamp")),col("status")).count().show()

+---------------------+------+-----+
dayofmonth(timestamp)|status|count|
+---------------------+------+-----+
 3| 404| 474|
 4| 200|63069|
 4| 404| 359|
 4| 302| 2513|
 3| 500| 53|
 1| 404| 316|
 3| 304| 4582|
 1| 200|58033|
 4| 304| 4507|
 2| 404| 291|
 1| 302| 2568|
 3| 200|81351|
 3| 302| 3122|
 2| 304| 3107|
 2| 403| 5|
 1| 304| 3797|
 2| 302| 2370|
 3| 403| 2|
 2| 200|54492|
 5| 404| 497|
+---------------------+------+-----+
only showing top 20 rows

In [0]:
logs_df1.groupBy(dayofmonth(col("timestamp")),col("status")).count().where(col("status")==404).show()

+---------------------+------+-----+
dayofmonth(timestamp)|status|count|
+---------------------+------+-----+
 3| 404| 474|
 4| 404| 359|
 1| 404| 316|
 2| 404| 291|
 5| 404| 497|
 6| 404| 640|
 7| 404| 570|
 8| 404| 300|
 10| 404| 398|
 11| 404| 471|
 9| 404| 348|
 13| 404| 532|
 12| 404| 471|
 16| 404| 257|
 15| 404| 254|
 14| 404| 413|
 17| 404| 406|
 18| 404| 465|
 20| 404| 428|
 19| 404| 639|
+---------------------+------+-----+
only showing top 20 rows